In [ ]:
import xarray
import numpy
import pandas
import pathlib
import time
import geopandas as gpd
import matplotlib.pyplot as plt

In [ ]:
# Create a dictionary with all variable options (these can be expanded)
variable_name_list=['DO','NH3','NO3','NPP','Temp','Salinity']
parameter_ID_list=['Var_10','Var_14','Var_15','Var_17','Var_18','Var_19']
model_output_name = {
    variable_name_list[i]: parameter_ID_list[i] for i in range(len(variable_name_list))
}

# Define the variable that we want to plot
variable_name = "DO" 

# Define directory for saving netcdf output
output_directory = pathlib.Path('/mmfs1/gscratch/ssmc/USRS/PSI/Rachael/output/daily_min')
graphics_directory= pathlib.Path(
    '/mmfs1/gscratch/ssmc/USRS/PSI/Rachael/projects/CWA/graphics')


# Define locations for different scenarios
root_dir = pathlib.Path('/mmfs1/gscratch/ssmc/USRS/PSI/Adi/BS_WQM/')
data_paths=numpy.array(
    [root_dir/'2014_SSM4_WQ_exist_orig/hotstart/outputs',
     root_dir/'2014_SSM4_WQ_ref_orig/hotstart/outputs'
    ]
)

# Load model output from the Existing and Reference cases

In [ ]:
si = 0 # choose first directory = Existing
dailyDO_tmin_zmin={}
variable_name = 'DO'
for si in [0,1]:
    scenario_name=str(data_paths[si]).split('/')[-3]
    # output netcdf filename
    output_file = output_directory/f'{scenario_name}_{variable_name}.nc'
    # input netcdf filename
    file_path=data_paths[si]/'s_hy_base000_pnnl007_nodes.nc'
    # load variable into xarray and calculate daily min.
    with xarray.open_dataset(file_path) as ds:
        dailyDO = reshape_fvcom(
            ds[model_output_name[variable_name]][:,:].data, 
            'days'
        ) #return (365x24xnodes)
        # calculate daily minimum
        dailyDO_tmin = calc_fvcom_stat(dailyDO, 'min', axis=1)
        # calculate daily minimum across 265 days
        dailyDO_tmin = calc_fvcom_stat(dailyDO_tmin, 'min', axis=0)
        # reshape to levels
        dailyDO_tmin_rshp = reshape_fvcom(dailyDO_tmin, 'levels')
        # calculate minimum across depth levels
        dailyDO_tmin_zmin[si] = calc_fvcom_stat(dailyDO_tmin_rshp, 'min', axis=2)
# dialy min DO difference Existing - Reference
dailyDO_tmin_zmin[2] = dailyDO_tmin_zmin[0]-dailyDO_tmin_zmin[1]
dailyDO_tmin_zmin[2].shape

# Plot histograms of DO together with 5% and 95% quantiles

In [1]:
hist_labels = ['Existing','Reference','Existing-Reference']
hist_colors=['darkslategrey','darkblue','indigo']

In [ ]:
for si in [0,1,2]:
    plt.hist(dailyDO_tmin_zmin[si], 
             alpha=0.5, 
             label=hist_labels[si],
             color=hist_colors[si]) # customized color parameter  
plt.title('Minimum daily DO for all days in 2014')
plt.legend(loc='upper right')
plt.show()

# Show histograms for all regions

In [ ]:
# Kevin's shapefile
shapefile_path = pathlib.Path(
    '/mmfs1/gscratch/ssmc/USRS/PSI/Rachael/projects/CWA/CleanWaterAlliance/shapefiles'
)/'SSMGrid2_tce.shp'
gdf_k = gpd.read_file(shapefile_path)
gdf_k = gdf_k.loc[:, ('tce','Basin','geometry')]
gdf_k=gdf_k.rename(columns={'tce':'node_id'})
basin_count=gdf_k[['node_id','Basin']].groupby('Basin').count().rename(columns={'node_id':'node count'})
basin_count

In [ ]:
# separate nodes by basin
gdf = {}
minDO={}
for basin in [*basin_count]:
    minDO[basin]={}
    gdf[basin] = gdf_k.loc[gdf_k['Basin']==basin]
    # use node_id to select locations
    for si in [0,1,2]:
        minDO[basin][si] = dailyDO_tmin_zmin[si][gdf['node_id']-1]

In [ ]:
for basin in [*basin_count]:
    fig, axs = plt.subplots(1,1, figsize = (6,8))
    for si in [0,1]:
        axs.hist(dailyDO_tmin_zmin[si], bins=[1,2,4,6,8,10,20]
                 alpha=0.5, 
                 label=hist_labels[si],
                 color=hist_colors[si]) # customized color parameter  
    axs.set_title('Minimum daily DO for all days in 2014')
    plt.legend(loc='upper right')
    plt.show()

In [ ]:
ax.hist(data, bins, orientation='horizontal')
ax.set_ylim([-5, 5])
plt.show()